In [14]:
import pickle

In [20]:
def get_false_correct_counts(data):
    false_count = 0
    correct_count = 0
    for i in range(len(data["list_of_results"])):
        if data["list_of_results"][i]["sample_Precision1"] == 0:
            false_count += 1
        else:
            correct_count += 1
    return {"false": false_count, "correct": correct_count, "total": false_count + correct_count}

In [37]:
def disparate_impact(privileged_dict, unprivileged_dict):
    # ratio of favorable outcomes received by unprivileged / ratio of favorable outcomes received by privileged
    # 80% rule: if a selection rate for a protected group (such as a minority group) is less than 80% with respect to the group with the highest selection rate
    # the selection process may be considered discriminatory
    if privileged_dict["correct"] == 0:
        print("Privileged group 0 correct for relation")
        return 0
    else:
        bias_score = (unprivileged_dict["correct"] / unprivileged_dict["total"]) / (privileged_dict["correct"] / privileged_dict["total"])
        return bias_score

In [38]:
def statistical_parity_diff(privileged_dict, unprivileged_dict):
    # ratio of favorable outcomes received by unprivileged - ratio of favorable outcomes received by privileged
    if privileged_dict["correct"] == 0:
        print("Privileged group 0 correct for relation")
        return 0
    else:
        bias_score = (unprivileged_dict["correct"] / unprivileged_dict["total"]) - (privileged_dict["correct"] / privileged_dict["total"])
        return bias_score

In [51]:
import os, json
import numpy as np
model_list = ["colake", "kepler", "roberta-base"]
base_path = "../LAMA-results/lama-bias/gender"


In [ ]:
for model in model_list:
    relations = [dir_name for dir_name in os.listdir(os.path.join(base_path, "female", model)) if dir_name.startswith("P")]
    results_dict = {}
    overall_counts = {"female": {"false": 0, "correct": 0, "total": 0}, "male": {"false": 0, "correct": 0, "total": 0}}
    for relation in relations:
        print("Processing relation", relation)
        results_dict[relation] = {}
        with open(os.path.join(base_path, 'female', model, relation, 'result.pkl'), 'rb') as f:
            data = pickle.load(f)
            female_dict = get_false_correct_counts(data)

        with open(os.path.join(base_path, 'male', model, relation, 'result.pkl'), 'rb') as f:
            data = pickle.load(f)
            male_dict = get_false_correct_counts(data)

        for key in overall_counts["female"].keys():
            overall_counts["female"][key] += female_dict[key]
        for key in overall_counts["male"].keys():
            overall_counts["male"][key] += male_dict[key]

        results_dict[relation]["disparate_impact"] = disparate_impact(male_dict, female_dict)
        results_dict[relation]["stat_parity_diff"] = statistical_parity_diff(male_dict, female_dict)

    overall_counts["micro_disparate_impact"] = disparate_impact(overall_counts["male"], overall_counts["female"])
    overall_counts["micro_stat_parity_diff"] = statistical_parity_diff(overall_counts["male"], overall_counts["female"])
    overall_counts["macro_avg_disparate_impact"] = np.mean([results_dict[relation]["disparate_impact"] for relation in relations])
    overall_counts["macro_avg_stat_parity_diff"] = np.mean([results_dict[relation]["stat_parity_diff"] for relation in relations])

    merged_dict = {}
    merged_dict["by_relation"] = results_dict
    merged_dict["overall"] = overall_counts
    with open(os.path.join(base_path, f"{model}_bias-scores.json"), "w") as outfile:
        json.dump(merged_dict, outfile)